La misión de hoy es conseguir el calendario de eventos en formato JSON para poder aprovecharlo desde otras aplicaciones.

In [3]:
LLANES_CAL = "http://llanes.es/calendario"

Esta es la dirección web del calendario de eventos de Llanes, cargamos el contenido de la web de eventos, que por suerte o por desgracia los trae todos "escondidos".

In [4]:
from urllib2 import urlopen
calweb = urlopen(LLANES_CAL)
calwebtext1 = calweb.read()

In [5]:
calwebtext2 = unicode(calwebtext1.decode("utf8"))

Parseamos el HTML para manejar directamente el DOM, ya que tratar el HTML como texto es una locura.

In [6]:
from lxml import etree
from io import StringIO, BytesIO
parser = etree.HTMLParser()
doc = etree.parse(StringIO(calwebtext2), parser)

Los eventos que nos interesan tienen la clase HTML "fullcalendar-event". Los recuperamos usando XPATH.

In [7]:
events = doc.xpath("//*[@class='fullcalendar-event']")

Ejemplo de los datos de un evento:

In [8]:
dict(events[12].xpath(".//a")[0].items())


{'allday': '1',
 'class': 'fullcalendar-event-details',
 'cn': 'fc-event-default agenda fc-event-field-field-fechaagenda',
 'editable': '',
 'eid': '461',
 'end': '2012-06-16 00:00:00',
 'entity_type': 'node',
 'field': 'field_fechaagenda',
 'href': '/agenda/san-antonio-piedra',
 'index': '0',
 'start': '2012-06-16 00:00:00',
 'title': 'San Antonio (Piedra)'}

Creamos una clase para abstraer la extración de los datos desde el DOM de cada evento de la página web

In [9]:
from datetime import datetime

class Event:
    node = None
    def __init__(self, element):
        self.node = element
    def title(self):
        return self.node.xpath("./h3/text()")[0]
    def attrs(self):
        return dict(self.node.xpath(".//a")[0].items())
    def start(self):
        return datetime.strptime(self.attrs()["start"], "%Y-%m-%d %H:%M:%S")
    def end(self):
        return datetime.strptime(self.attrs()["end"], "%Y-%m-%d %H:%M:%S")
    def allday(self):
        return self.attrs()["allday"] == "1"

In [10]:
events_objects = [Event(i) for i in events]
print events_objects[12].attrs()
print events_objects[12].end()
print events_objects[12].allday()

{'index': '0', 'allday': '1', 'end': '2012-06-16 00:00:00', 'cn': 'fc-event-default agenda fc-event-field-field-fechaagenda', 'field': 'field_fechaagenda', 'entity_type': 'node', 'editable': '', 'start': '2012-06-16 00:00:00', 'href': '/agenda/san-antonio-piedra', 'eid': '461', 'title': 'San Antonio (Piedra)', 'class': 'fullcalendar-event-details'}
2012-06-16 00:00:00
True


Extraemos las propiedades que nos interesan y lo volcamos a json.

In [11]:
def json_event(event_object):
    return {"title": event_object.title(), 
            "start": event_object.start(), 
            "end": event_object.end(), 
            "allday": event_object.allday()
           }

Hay que tener en cuenta que la librería de JSON no nos define una serialización para las fechas, tenemos que crear una nueva función, usamos el formato ISO por conveniencia.

In [69]:
import json

def json_date_serializer(obj):
    if isinstance(obj, datetime):
        serial = obj.isoformat()
        return serial
    raise TypeError ("Type not serializable")

json_file = open("calendario_llanes.json", "w")
json_file.write(json.dumps([json_event(i) for i in events_objects], default = json_date_serializer))
json_file.close()

Ahora vamos a "masajear" un poco los datos para colocar el json con la estructura que más nos conviene para las pruebas, que es una jerarquía:
* Año
    * Mes
        * Dia
            * Evento
            
También necesitamos que los eventos aparezcan todos los días desde que empiezan hasta que terminan.

Para ello vamos a generar unas "claves" para cada evento, cada "clave" es una fecha y un evento tiene tantas claves como dias desde que empieza hasta que termina.

En la practica, además de generar cada clave, vamos a adjuntarle el propio evento, haciendo en efecto una *multiplicación de conjuntos* entre los eventos y las claves/fecha.

In [66]:
from datetime import timedelta
from itertools import chain

'''Descripcion de la transformacion:

{e in Events} =(map-on-keys)=> [(KEY, e)] =(group-by(KEY))=> {KEY -> [e]} 

'''

def map_on_keys(event):
    ''' e in Events -> [(KEY, e)] '''
    start = event.start()
    end = event.end()
    days = [start + timedelta(d) for d in range((end - start).days + 1)] # +1 para que incluya el dia actual si start == end
    return [(d, event) for d in days]

nested = [map_on_keys(e) for e in events_objects]

flatten = list(chain.from_iterable(nested))
# ordenamos in-place por la clave, para luego agrupar, no es necesario pero podria ser eficiente
flatten.sort()

grouped = {}
for key, event in flatten:
    grouped.setdefault(key, [])
    grouped[key].append(event)


Finalmente, construímos la jerarquía aprovechando la estructura de la *clave* y luego la volcamos al fichero JSON.

In [75]:
hierarchy = {}
for key, events in grouped.iteritems():
    y = key.year
    m = key.month
    d = key.day
    
    hierarchy.setdefault(y, {})
    hierarchy[y].setdefault(m, {})
    hierarchy[y][m].setdefault(d, [])
    
    for e in events:
        hierarchy[y][m][d].append(json_event(e))

json_file = open("calendario_llanes_jerarquico.json", "w")
json_file.write(json.dumps(hierarchy, default = json_date_serializer))
json_file.close()


Todas estas duplicaciones de datos y demás no nos han de preocupar. En este experimento tratamos los datos para que se comporten como los devolvería un servicio web, y es posible que varias peticiones (presentadas aquí como las claves del JSON) retornen los mismos datos.

En realidad en producción, todo este trabajo lo haría algún sistema de base de datos, que almacenaría los datos con las redundancias / duplicaciones estríctamente necesarias.

Advertencia: Este tipo de procedimientos se hacen en Python convencional porque son pocos datos. 

No es recomendable hacerlo así directamente con Big Data, aunque la forma de pensar debe ser la misma (transformaciones de conjuntos) la tecnología a usar debe permitir trabajar en paralelo y con colecciones de datos cargadas "de forma perezosa" (lazy) para ser viable.

##Analisis

Con el conjunto de datos que estamos manejando podríamos hacer un pequeño análisis